# COGS 108 - Final Project 

# Overview

*Fill in your overview here*

# Names

- Les Wang
- Sebastian Troncoso
- Tamuz Hod
- Josh Freilich
- Baizhou Xu

# Group Members IDs

- A12985002
- A14217903
- A15230380
- A13837230
- U08254404

# Research Question

How does the type of business affect the type of crime in the area?

## Background and Prior Work

*Fill in your background and prior work here* 

References (include links):
- 1)
- 2)

# Hypothesis


We predict that the more family friendly businesses in an area reduces the crime rate in the neighborhood whereas the less family friendly businesses will increase the crime rate.

# Dataset(s)

*Fill in your dataset information here*

(Copy this information for each dataset)
- Dataset Name:
- Link to the dataset:
- Number of observations:

1-2 sentences describing each dataset. 

If you plan to use multiple datasets, add 1-2 sentences about how you plan to combine these datasets.

# Setup

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.neighbors import kneighbors_graph
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report, precision_recall_fscore_support

# Data Cleaning

In this step the datasets are imported and a universal identifier (uid) is created from the stop_id and pid columns.

In [16]:
reason_df = pd.read_csv("Datasets/reason_for_stop.csv")
reason_df['uid'] = reason_df.stop_id.astype(str).str.cat(reason_df.pid.astype(str), sep='_')
details_df = pd.read_csv("Datasets/basic_details.csv",low_memory=False)
details_df['uid'] = details_df.stop_id.astype(str).str.cat(details_df.pid.astype(str), sep='_')


Next the datasets are merged on the <b>uid</b> and all columns that are not relevant to this analysis are dropped.

In [28]:
totalStop_df = pd.merge(reason_df, details_df, on='uid')
totalStop_df = totalStop_df.drop(["agency", "beat", "beat_name", "isschool","school_name", "isstudent"], axis=1)

,stop_id_x,pid_x,reason_for_stop,reason_for_stopcode,reason_for_stop_code_text,reason_for_stop_detail,reason_for_stop_explanation,uid,stop_id_y,ori,...,highway_exit,cityname,pid_y,perceived_limited_english,perceived_age,perceived_gender,gender_nonconforming,gend,gend_nc,perceived_lgbt
0,2443,1,Reasonable Suspicion,64005,647(F) PC - DISORD CONDUCT:ALCOHOL (M) 64005,Officer witnessed commission of a crime,"staggering, unable to safely walk",2443_1,2443,CA0371100,...,NaN,SAN DIEGO,1,0,25,Male,0,1,NaN,No
1,2444,1,Traffic Violation,54106,22350 VC - UNSAFE SPEED:PREVAIL COND (I) 54106,Moving Violation,Speeding,2444_1,2444,CA0371100,...,NaN,SAN DIEGO,1,0,25,Male,0,1,NaN,No
2,2447,1,Reasonable Suspicion,53072,415(1) PC - FIGHT IN PUBLIC PLACE (M) 53072,Matched suspect description,Both parties involved in argument.,2447_1,2447,CA0371100,...,NaN,SAN DIEGO,1,0,30,Male,0,1,NaN,No
3,2447,2,Reasonable Suspicion,53072,415(1) PC - FIGHT IN PUBLIC PLACE (M) 53072,Other Reasonable Suspicion of a crime,Both parties engaged in argument.,2447_2,2447,CA0371100,...,NaN,SAN DIEGO,2,0,30,Female,0,2,NaN,No
4,2448,1,Traffic Violation,54106,22350 VC - UNSAFE SPEED:PREVAIL COND (I) 54106,Moving Violation,UNSAFE DRIVING,2448_1,2448,CA0371100,...,NaN,SAN DIEGO,1,0,23,Male,0,1,NaN,No


,stop_id,exp_years,stopduration,stop_in_response_to_cfs,officer_assignment_key,block,isschool,beat,pid,isstudent,perceived_limited_english,perceived_age,gender_nonconforming,gend,gend_nc
count,89485.000000,89485.000000,89485.000000,89485.000000,89485.000000,8.147600e+04,89485.000000,89485.000000,89485.000000,89485.000000,89485.000000,89485.000000,89485.000000,89485.000000,83.0
mean,43098.030541,7.559725,26.878237,0.126669,1.618126,4.507127e+03,0.001185,504.594636,1.166877,0.000793,0.034229,37.047092,0.000928,1.281354,5.0
std,23598.601930,7.634611,51.262885,0.332604,2.145314,8.506998e+04,0.034397,245.744360,0.631672,0.028157,0.181818,13.420349,0.030441,0.463033,0.0
min,2443.000000,1.000000,1.000000,0.000000,1.000000,0.000000e+00,0.000000,111.000000,1.000000,0.000000,0.000000,1.000000,0.000000,0.000000,5.0
25%,22775.000000,2.000000,8.000000,0.000000,1.000000,1.200000e+03,0.000000,314.000000,1.000000,0.000000,0.000000,25.000000,0.000000,1.000000,5.0
50%,42848.000000,4.000000,12.000000,0.000000,1.000000,3.000000e+03,0.000000,521.000000,1.000000,0.000000,0.000000,35.000000,0.000000,1.000000,5.0
75%,62664.000000,11.000000,25.000000,0.000000,1.000000,4.500000e+03,0.000000,712.000000,1.000000,0.000000,0.000000,47.000000,0.000000,2.000000,5.0
max,84525.000000,50.000000,1440.000000,1.000000,10.000000,9.999900e+06,1.000000,999.000000,20.000000,1.000000,1.000000,120.000000,1.000000,4.000000,5.0


# Data Analysis & Results

Include cells that describe the steps in your data analysis.

In [5]:
## YOUR CODE HERE
## FEEL FREE TO ADD MULTIPLE CELLS PER SECTION

# Ethics & Privacy

*Fill in your ethics & privacy discussion here*

# Conclusion & Discussion

*Fill in your discussion information here*